# Downloading dataset

In [1]:
!curl -O https://storage.googleapis.com/wandb_datasets/nature_12K.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3639M  100 3639M    0     0   161M      0  0:00:22  0:00:22 --:--:--  161M 65.1M    0     0  93.0M      0  0:00:39 --:--:--  0:00:39 92.9M


## 